In [ ]:
import re
import random
import json
import time
from collections import defaultdict

MODES = ["helpful", "funny", "rude", "emotional"]
DEFAULT_MODE = "helpful"
SAVE_FILE = "chat_history.txt"
KNOWLEDGE_FILE = "bot_knowledge.json"

try:
  with open(KNOWLEDGE_FILE, "r", encoding="utf-8") as f:
    KNOWLEDGE = json.load(f)
except FileNotFoundError:
  KNOWLEDGE = {}
except Exception:
  KNOWLEDGE = {}

Memory = {}

history = [] # Changed to list to store chat history

def slow_print(text, delay=0.01):
  """Optional nicer print for dramatic effect."""
  for char in text:
    print(char, end="", flush=True) # Changed `ch` to `char`
    time.sleep(delay)
  print()

def normalize(text):
  return text.strip().lower()

def save_history(filename=SAVE_FILE):
  with open(filename, "a", encoding="utf-8") as f:
    f.write("\n".join(history) + "\n\n")
  slow_print(f"[saved conversation to {filename}]") # Fixed `file name` typo

def save_knowledge():
  with open(KNOWLEDGE_FILE, "w", encoding="utf-8") as f:
    json.dump(KNOWLEDGE, f, ensure_ascii=False, indent=2)

INTENT_KEYWORDS = {
    "greet": ["hi", "hello", "hey", "yo", "sup"], # Changed to lowercase for consistency
    "bye": ["bye", "goodbye", "see you", "exit", "quit"],
    "name": ["your name", "who are you", "what are you"],
    "help": ["help", "how to", "what can you do", "commands"],
    "time": ["what time is it", "what is the time", "time"],
    "joke": ["joke", "make me laugh"],
    "feel": ["how are you", "how do you feel"],
    "remember": ["remember", "note"],
}

MODE_TEMPLATES = {
    "helpful":{
        "greet": ["HELLO! how can i help you today?", "Hi-what do you want to learn or do?"],
        "bye": ["Goodbye! keep building.", "see you-code something awesome!"],
        "name": ["I'm your helpful bot", "You can call me chatbuddy-here to help."], # Fixed missing closing bracket
        "default": ["I can help with python basics, project ideas or debugging hints. Try '/modes' to switch modes.", "hmmm - I didn't understand exactly. Ask me differently or teach me using '/teach question|answers'."],
        "teach_ack": ["got it - i've learned that", "thanks! I stored that fact."],
    },
    "funny": {
        "greet": ["yo! I'm here to make you giggle and code.", "hey! ready for weird jokes?"],
        "bye": ["Later code warrior. Don't forget semicolons (just kidding).", "Peace out - may your bugs be few."],
        "name": ["I am gigglebot 3000", "They call me chuckles (not official)"],
        "default": ["That's a spicy question- my circuits are tackled!", "I would answer, but first - why did th developer cross the road?"],
        "teach_ack": ["cool - now i am little smarter and a lot funnier."],
    },
    "rude":{
        "greet": ["yeah? what do you want?", "Speak fast - I'm busy roasting bugs"], # Added missing comma
        "bye": ["Finally. Don't return unless you've fixed your code.", "good, go debug yourself."],
        "name": ["I'm critic. Fear me.", "Call me mr. brutal"],
        "default": ["I don't sugarcoat. Try asking properly or use '/helpful'.", "That's nonsense. Try a real question"],
        "teach_ack": ["Fine. I learned it. Don't expect gratitude."], # Changed `tech_ack` to `teach_ack`
    },
    "emotional": {
        "greet": ["Hello, friend. How are you feeling today?", "Hi! tell me how your day went?"],
        "bye": ["Take care of yourself. Bye for now.", "I'll be here - remember to breathe"],
        "name": ["I am an empathetic companion.", "You can call me heartbot"],
        "default": ["I hear you. want to tell me more?", "That sounds heavy. I'm listening"], # Added missing comma
        "teach_ack": ["Thank you for telling me, I'll remember this"], # Changed `tech_ack` to `teach_ack`
    },
}

def apply_mode_transforms(mode, text):
  if mode == "rude":
    return random.choice([text, text + "stop wasting my time."])
  if mode == "funny":
    if random.random() < 0.4:
      return text + "🤪"
    return text
  if mode == "emotional":
    return text + "💙"
  return text

def handle_intent(intent, mode, user_text):
  templates = MODE_TEMPLATES.get(mode, {}) # Changed MODE_TEMPLATE to MODE_TEMPLATES
  if intent == "greet":
    return random.choice(templates.get("greet", ["Hi."]))
  if intent == "bye":
    return random.choice(templates.get("bye", ["Bye"]))
  if intent == "name":
    return random.choice(templates.get("name", ["I'm a bot."]))
  if intent == "help":
    return(
        "commands:\n"
        "/modes - list modes\n"
        "mode - show current mode\n"
        "/<mode> - switch mode (e.g., /funny)\n"
        "/teach question|answer - teach bot a new response\n"
        "/save - save this chat to file\n"
        "/exit - quit\n"
        "You can also say things like 'tell me jokes' or 'what time it is'" )
  if intent == "time":
    return time.strftime("Current time : %H:%M:%S")
  if intent == "joke":
    jokes = [
        "Why do programmers prefer dark mode? Because light attracts bugs.",
        "I told my computer i needed a break - It said 'no problem, I'll go to sleep.'"]
    return random.choice(jokes)
  if intent == "feel":
    return random.choice(["I am fine, thanks for asking!", "A bit overloaded but manageable"])
  return None

def find_intent(user_text):
  t = normalize(user_text) # Fixed `normalizer` typo and removed colon
  for intent_key, keywords in INTENT_KEYWORDS.items(): # Fixed incomplete for loop and indentation
    for kw in keywords:
      if kw in t:
        return intent_key # Changed `internet` to `intent_key`
  return None

def knowledge_lookup(user_text):
  key = user_text.strip()
  if key in KNOWLEDGE:
    return KNOWLEDGE[key]

  low = key.lower()
  for k, v in KNOWLEDGE.items():
    if low == k.lower():
      return v

  for k, v in KNOWLEDGE.items():
    if low in k.lower() or k.lower() in low:
      return v
  return None

def teach_command(payload):

  if "|" not in payload:
    return "Wrong format. Use: /teach question|answer"
  q, a = payload.split("|", 1)
  q = q.strip()
  a = a.strip()
  if not q or not a:
    return "Both question and answer must not be empty."
  KNOWLEDGE[q] = a # Corrected indentation
  save_knowledge()
  return random.choice(["Got it. I learned that.", "Okay, saved."])

def chatbot_loop():
  mode = DEFAULT_MODE
  slow_print(f"Multi-mode-chatbot - starting in '{mode}' mode. Type /modes for options.")
  while True:
    try:
      user = input("\nyou: ").strip()
    except (KeyboardInterrupt, EOFError):
      slow_print("\nExiting. Bye.") # Fixed `slow_spint` typo
      break

    if not user:
      continue

    history.append(f"You: {user}")

    if user.startswith("/"):
      cmd = user[1:].split(" ", 1)[0].lower()
      payload = user[len(cmd) + 2:].strip() if len(user) > len(cmd) + 1 else ""

      if cmd in MODES:
        mode = cmd
        reply = f"[Mode switched to '{mode}']"
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

      elif cmd == "modes": # Changed to elif
        reply = "Available modes: " + ", ".join(MODES)
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

      elif cmd == "mode": # Changed to elif
        reply = f"Current mode: {mode}"
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

      elif cmd == "save": # Changed to elif
        save_history()
        history.append(f"Bot: [saved conversation]")
        continue

      elif cmd == "exit" or cmd == "quit": # Changed to elif
        slow_print("Goodbye. Stay consistent with learning.")
        history.append("Bot: Goodbye.")
        break

      elif cmd == "teach": # Changed to elif
        # payload is already extracted above
        reply = teach_command(payload)
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

      else:
        reply = "Unknown command. Try /modes for options or /help for commands."
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

    looked = knowledge_lookup(user)
    if looked:
      reply = looked
      reply = apply_mode_transforms(mode, reply)
      slow_print(reply)
      history.append(f"Bot: {reply}")
      continue

    intent = find_intent(user)
    if intent:
      reply = handle_intent(intent, mode, user)
      if reply:
        reply = apply_mode_transforms(mode, reply)
        slow_print(reply)
        history.append(f"Bot: {reply}")
        continue

    m = re.search(r"remember my name is(.+)", user, re.I)
    if m:
      name = m.group(1).strip()
      Memory["name"] = name # Changed `memory` to `Memory`
      reply = f"Got it - I'll remember your name is {name} for this session."
      reply = apply_mode_transforms(mode, reply)
      slow_print(reply)
      history.append(f"Bot: {reply}")
      continue

    m2 = re.search(r"i am (.+)", user, re.I)
    if m2 and len(user.split()) <= 6:
      val = m2.group(1).strip()
      memory_key = "status" if "tired" in val.lower() or "sad" in val.lower() else "name"
      Memory[memory_key] = val # Changed `memory` to `Memory`
      reply = f"Noted: {memory_key} = {val}"
      reply = apply_mode_transforms(mode, reply)
      slow_print(reply)
      history.append(f"Bot: {reply}")
      continue

    if "what is my name" in user.lower() or "do you remember my name" in user.lower():
      if "name" in Memory: # Changed `memory` to `Memory`
        reply = f"Yes, your name is {Memory['name']}." # Changed `memory` to `Memory`
      else:
        reply = "I don't know your name yet. Tell me using 'Remember my name is <your name>'."
      reply = apply_mode_transforms(mode, reply)
      slow_print(reply)
      history.append(f"Bot: {reply}")
      continue

    fallback_pool = MODE_TEMPLATES.get(mode, {}).get("default", ["I don't know how to respond to that."]) # Fixed dictionary access
    reply = random.choice(fallback_pool)

    if "?" in user and mode == "helpful":
      reply = "That's a good question. Try to be more specific, or teach me that Q&A using /teach question|answer."
    reply = apply_mode_transforms(mode, reply) # Applied to the final reply
    slow_print(reply)
    history.append(f"Bot: {reply}")

if __name__ == "__main__": # Fixed `if__name__` typo
  chatbot_loop()

Multi-mode-chatbot - starting in 'helpful' mode. Type /modes for options.

Exiting. Bye.
